In [34]:
# Derived partially from
# - https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
import os
import optuna
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from distributed import wait

print(f"Cuda is available: {torch.cuda.is_available()}")

# Root directory for dataset
dataroot = "data/celeba"

# Number of workers for dataloader
workers = 0  # IMPORTANT w/ optuna; it launches a daemonic process, so PyTorch can't itself use it then.

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 20

# Beta1 hyperparameter for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1


# We create a fake image dataset. This ought to be replaced by
# your actual dataset or pytorch's example datasets. We do this here
# to focus more on computation than actual convergence.
class FakeImageDataset(torch.utils.data.Dataset):
    def __init__(self, count=1_000):
        self.labels = np.random.randint(0, 5, size=count)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        label = self.labels[idx]
        torch.random.seed = label
        img = torch.rand(3, image_size, image_size)
        return img, label

dataset = FakeImageDataset()

# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
        
class Generator(nn.Module):
    def __init__(self, ngpu, activation=nn.Sigmoid()):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. ``(ngf*8) x 4 x 4``
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. ``(ngf*4) x 8 x 8``
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. ``(ngf*2) x 16 x 16``
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. ``(ngf) x 32 x 32``
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            activation
            # state size. ``(nc) x 64 x 64``
        )

    def forward(self, input):
        return self.main(input)

    @classmethod
    def from_trial(cls, trial):
        activations = [nn.Sigmoid, nn.Tanh]
        idx = trial.suggest_categorical("generator_activation", list(range(len(activations))))
        activation = activations[idx]()
        return cls(ngpu, activation)

class Discriminator(nn.Module):
    def __init__(self, ngpu, leaky_relu_slope=0.2, activation=nn.Sigmoid()):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is ``(nc) x 64 x 64``
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(leaky_relu_slope, inplace=True),
            # state size. ``(ndf) x 32 x 32``
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(leaky_relu_slope, inplace=True),
            # state size. ``(ndf*2) x 16 x 16``
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(leaky_relu_slope, inplace=True),
            # state size. ``(ndf*4) x 8 x 8``
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(leaky_relu_slope, inplace=True),
            # state size. ``(ndf*8) x 4 x 4``
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            activation
        )

    def forward(self, input):
        return self.main(input)
    
    @classmethod
    def from_trial(cls, trial):
        activations = [nn.Sigmoid, nn.Tanh]
        idx = trial.suggest_categorical("descriminator_activation", list(range(len(activations))))
        activation = activations[idx]()
        
        slopes = np.arange(0.1, 0.4, 0.1)
        idx = trial.suggest_categorical("descriminator_leaky_relu_slope", list(range(len(slopes))))
        leaky_relu_slope = slopes[idx]
        return cls(ngpu, leaky_relu_slope, activation)
        

def objective(trial):
    # Decide which device we want to run on
    device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

    ############################
    ### Create the generator ###
    netG = Generator.from_trial(trial).to(device)

    # Handle multi-GPU if desired
    if (device.type == 'cuda') and (ngpu > 1):
        netG = nn.DataParallel(netG, list(range(ngpu)))

    # Apply the ``weights_init`` function to randomly initialize all weights
    #  to ``mean=0``, ``stdev=0.02``.
    netG.apply(weights_init)

    ################################
    ### Create the Discriminator ###
    netD = Discriminator.from_trial(trial).to(device)

    # Handle multi-GPU if desired
    if (device.type == 'cuda') and (ngpu > 1):
        netD = nn.DataParallel(netD, list(range(ngpu)))

    # Apply the ``weights_init`` function to randomly initialize all weights
    # like this: ``to mean=0, stdev=0.2``.
    netD.apply(weights_init)

    ############################################
    ### Remaining crierion, optimizers, etc. ###
    # Initialize the ``BCELoss`` function
    criterion = nn.BCELoss()

    # Create batch of latent vectors that we will use to visualize
    #  the progression of the generator
    fixed_noise = torch.randn(64, nz, 1, 1, device=device)

    # Establish convention for real and fake labels during training
    real_label = 1.
    fake_label = 0.
    
    # Learning rate for optimizers
    lr = 0.00001

    # Setup Adam optimizers for both G and D
    optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
    optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))
    
    #####################
    ### Training Loop ###

    # Lists to keep track of progress
    img_list = []
    G_losses = []
    D_losses = []
    iters = 0

    print("Starting Training Loop...")
    # For each epoch
    for epoch in range(num_epochs):
        # For each batch in the dataloader
        for i, data in enumerate(dataloader, 0):

            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            ###########################
            ## Train with all-real batch
            netD.zero_grad()
            # Format batch
            real_cpu = data[0].to(device)
            b_size = real_cpu.size(0)
            label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
            # Forward pass real batch through D
            output = netD(real_cpu).view(-1)
            # Calculate loss on all-real batch
            errD_real = criterion(output, label)
            # Calculate gradients for D in backward pass
            errD_real.backward()
            D_x = output.mean().item()

            ## Train with all-fake batch
            # Generate batch of latent vectors
            noise = torch.randn(b_size, nz, 1, 1, device=device)
            # Generate fake image batch with G
            fake = netG(noise)
            label.fill_(fake_label)
            # Classify all fake batch with D
            output = netD(fake.detach()).view(-1)
            # Calculate D's loss on the all-fake batch
            errD_fake = criterion(output, label)
            # Calculate the gradients for this batch, accumulated (summed) with previous gradients
            errD_fake.backward()
            D_G_z1 = output.mean().item()
            # Compute error of D as sum over the fake and the real batches
            errD = errD_real + errD_fake
            # Update D
            optimizerD.step()

            ############################
            # (2) Update G network: maximize log(D(G(z)))
            ###########################
            netG.zero_grad()
            label.fill_(real_label)  # fake labels are real for generator cost
            # Since we just updated D, perform another forward pass of all-fake batch through D
            output = netD(fake).view(-1)
            # Calculate G's loss based on this output
            errG = criterion(output, label)
            # Calculate gradients for G
            errG.backward()
            D_G_z2 = output.mean().item()
            # Update G
            optimizerG.step()

            # Output training stats
            if i % 50 == 0:
                print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                      % (epoch, num_epochs, i, len(dataloader),
                         errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

            # Save Losses for plotting later
            G_losses.append(errG.item())
            D_losses.append(errD.item())

            # Check how the generator is doing by saving G's output on fixed_noise
            if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
                with torch.no_grad():
                    fake = netG(fixed_noise).detach().cpu()
                img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

            iters += 1

        # Report to Optuna
        trial.report(errD.item(), epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return errD.item()

Cuda is available: True


### Start Coiled cluster using package_sync w/ GPU workers

In [2]:
import coiled

cluster = coiled.Cluster(
    name="pytorch-optuna-milesg1",
    n_workers=8,
    worker_gpu=True,
    worker_options={"nthreads": 1},
)

client = cluster.get_client()

Output()

jupyter-server-ydoc 0.1.4 has requirement jupyter-ydoc==0.1.13, but you have jupyter-ydoc 1.0.1.

coiled-runtime 0.2.1 has requirement cloudpickle==2.2.1, but you have cloudpickle 2.2.0.

coiled-runtime 0.2.1 has requirement dask==2023.3.2, but you have dask 2022.11.1.

coiled-runtime 0.2.1 has requirement dask-labextension==6.1.0, but you have dask-labextension 6.0.0.

...

coiled-runtime 0.2.1 has requirement zarr==2.14.2, but you have zarr 2.13.3.

Package - jupyter-server-ydoc, Pip check had the following issues that need resolving: 
jupyter-server-ydoc 0.1.4 has requirement jupyter-ydoc==0.1.13, but you have jupyter-ydoc 1.0.1.

╭──────────────────────────────────── Not Synced with Cluster ─────────────────────────────────────╮
│                       ╷                                                              ╷           │
│   Package             │ Error                                                        │ Risk      │
│ ╶─────────────────────┼──────────────────────────────────────────────────────────────┼─────────╴ │
│   jupyter-server-ydoc │ Pip check had the following issues that need resolving:      │ Warning   │
│                       │ jupyter-server-ydoc 0.1.4 has requirement                    │           │
│                       │ jupyter-ydoc==0.1.13, but you have jupyter-ydoc 1.0.1.       │           │
│   coiled-runtime      │ Pip check had the following issues that need resolving:      │           │
│                       │ coiled-runtime 0.2.1 has requirement cloudpickle==2.2.1, but │           │
│                       │ you have cloudpickle 2.2.0.                                  │           │
│                       │ coiled-runtime 0.2.1 has requirement dask==2023.3.2, but you │           │
│                       │ have dask 2022.11.1.                                         │           │
│                       │ coiled-runtime 0.2.1 has requirement                         │           │
│                       │ dask-labextension==6.1.0, but you have dask-labextension     │           │
│                       │ 6.0.0.                                                       │           │
│                       │ coiled-runtime 0.2.1 has requirement dask-ml==2023.3.24, but │           │
│                       │ you have dask-ml 2022.5.27.                                  │           │
│                       │ coiled-runtime 0.2.1 has requirement distributed==2023.3.2,  │           │
│                       │ but you have distributed 2022.11.1.                          │           │
│                       │ coiled-runtime 0.2.1 has requirement fsspec==2023.3.0, but   │           │
│                       │ you have fsspec 2022.11.0.                                   │           │
│                       │ coiled-runtime 0.2.1 has requirement gcsfs==2023.3.0, but    │           │
│                       │ you have gcsfs 2022.11.0.                                    │           │
│                       │ coiled-runtime 0.2.1 has requirement ipywidgets==8.0.4, but  │           │
│                       │ you have ipywidgets 7.7.2.                                   │           │
│                       │ coiled-runtime 0.2.1 has requirement jupyterlab==3.6.2, but  │           │
│                       │ you have jupyterlab 3.5.0.                                   │           │
│                       │ coiled-runtime 0.2.1 has requirement lz4==4.3.2, but you     │           │
│                       │ have lz4 4.0.2.                                              │           │
│                       │ coiled-runtime 0.2.1 has requirement msgpack==1.0.5, but you │           │
│                       │ have msgpack 1.0.4.                                          │           │
│                       │ coiled-runtime 0.2.1 has requirement numba==0.56.4, but you  │           │
│                       │ have numba 0.56.3.                                           │           │
│                       │ coiled-runtime 0.2.1 has requirement pandas==1.5.3, but you  │           │
│                       │ have pandas 1.5.2.                                           │           │
│                       │ coiled-runtime 0.2.1 has requirement pyarrow==11.0.0, but    │           │
│                       │ you have pyarrow 10.0.1.                                     │           │
│                       │ coiled-runtime 0.2.1 has requirement pynvml==11.5.0, but you │           │
│                       │ have pynvml 11.4.1.                                          │           │
│                       │ coiled-runtime 0.2.1 has requiremen

Output()

/home/milesg/mambaforge/envs/coiled-benchmarks/lib/python3.10/site-packages/distributed/client.py:1361: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| pandas  | 1.5.2  | 1.5.3     | 1.5.3   |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


### TODO: With package_sync, it won't install pytorch with Cuda, so we need to re-install it on the workers


In [ ]:
from distributed import PipInstall

plugin = PipInstall(['torch', 'torchvision', 'torchaudio'], pip_options=['--force-reinstall'])
client.register_worker_plugin(plugin)

workers = list(client.scheduler_info()['workers'].keys())
client.restart_workers(workers)

### Perform hyperparameter tuning using Dask and Distributed on Coiled

In [39]:
from optuna.integration.dask import DaskStorage

study = optuna.create_study(direction='minimize', storage=DaskStorage(client=client))
n_trials = 8  # Set to your heart's desire, patience and cluster size. :)

jobs = [
    client.submit(study.optimize, objective, n_trials=1, pure=False)
    for _ in range(n_trials)
]
wait(jobs)

/tmp/ipykernel_11922/3321613542.py:3: ExperimentalWarning: DaskStorage is experimental (supported from v3.1.0). The interface can change in the future.
  study = optuna.create_study(direction='minimize', storage=DaskStorage(client=client))


DoneAndNotDoneFutures(done={<Future: finished, type: NoneType, key: optimize-0a587d14-3cca-46d3-93cd-83764967387c>, <Future: finished, type: NoneType, key: optimize-54ae592c-3000-49aa-a27c-563088558913>, <Future: finished, type: NoneType, key: optimize-36739789-28f0-469f-84f1-a95d3fab7128>, <Future: finished, type: NoneType, key: optimize-e36b6a72-d5d7-40b0-9f40-119d3e011ab1>, <Future: finished, type: NoneType, key: optimize-d5e36dfd-9b1b-40ba-9a65-adf7f25ab0cb>, <Future: finished, type: NoneType, key: optimize-8a7b80c1-0ef3-4814-8774-7446550e5c54>, <Future: finished, type: NoneType, key: optimize-2de9015b-508d-410e-a53e-60e6fa9031f4>, <Future: finished, type: NoneType, key: optimize-b32c75b5-7c48-442c-ba7b-d6e623e361dd>}, not_done=set())

In [37]:
from optuna.integration.dask import DaskStorage

study = optuna.create_study(direction='minimize', storage=DaskStorage(client=client))
client.submit(study.optimize, objective, n_trials=1, pure=False).result()

/tmp/ipykernel_11922/2169753959.py:3: ExperimentalWarning: DaskStorage is experimental (supported from v3.1.0). The interface can change in the future.
  study = optuna.create_study(direction='minimize', storage=DaskStorage(client=client))


In [38]:
client.restart()

<Client: 'tls://10.0.30.255:8786' processes=7 threads=7, memory=103.02 GiB>

In [40]:
client.shutdown()

2023-04-14 11:51:20,314 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
